## COVID-19 Interactive Analysis Dashboard
### What is COVID-19
* COVID-19 is a disease caused by a virus called **SARS-CoV-2**.
* Most people with COVID-19 have mild symptoms, but some people can become severely ill.
* Although most people with COVID-19 get better within weeks of illness, some people experience post-COVID conditions.
* Post-COVID conditions are a wide range of new, returning, or ongoing health problems people can experience more than four weeks after first being infected with the virus that causes COVID-19.
* Older people and those who have certain underlying medical conditions are more likely to get severely ill from COVID-19.
* Vaccines against COVID-19 are safe and effective.<br><br>
**Link** : https://www.cdc.gov/coronavirus/2019-ncov/faq.html

#### This Dashboard Provides an Overall View Of the  COVID-19 Status in the World

**Datasets Taken From :** <a href = "https://github.com/CSSEGISandData/COVID-19">COVID-19 Data Repository by the Center for Systems Science and Engineering (CSSE) at Johns Hopkins University</a>

In [1]:
#Import Libraries
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from __future__ import print_function
from ipywidgets import interact , interactive , fixed , interact_manual
import ipywidgets as widgets
import folium

In [2]:
death_df = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv")
confirmed_df = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")
recovered_df = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv")
country_df = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv")

In [3]:
country_df = country_df.rename({'Province_State' : 'Province State' ,'Country_Region' : 'Country Region' , 
                     'Last_Update' : 'Last Update' , 'Long_' : 'Long' ,
                    'People_Hospitalized' : 'People Hospitalized'  , 'Incident_Rate' : 'Incident Rate',
                     'Mortality_Rate' :'Mortality Rate' } , axis='columns' )
country_df = country_df.drop('Recovered' , axis = 1)

## Worst Hit Countries Based On Confirmed Cases

In [4]:
#Worst Hit Countries
worst_hit_df = country_df.sort_values(by = 'Confirmed' , ascending = False)
def highlight_cols(x):
    #variables store the colors
    r = 'background-color : red'
    p = 'background-color : lightpink'
    g = 'background-color : grey'
    #worst_hit_df will be a new dataframe that will have the index of country_df and columns of country_df
    worst_hit_df = pd.DataFrame('' , index = x.index , columns = x.columns)
    #Choosing the Index to highlight
    worst_hit_df.iloc[: , 4] = r
    worst_hit_df.iloc[: , 5] = g
    return worst_hit_df
#Apply this style on the country_df
def headval(Enter):
    Enter = int(Enter)
    return country_df.sort_values(by = 'Confirmed' , ascending = False).head(Enter).style.apply(highlight_cols , axis = None)
interact(headval , Enter = '5')

interactive(children=(Text(value='5', description='Enter'), Output()), _dom_classes=('widget-interact',))

<function __main__.headval(Enter)>

## Worst Hit Countries Based On Confirmed Cases (Scatter Plot)

In [5]:
def worst_hit_scatter(n):
    fig = px.scatter(worst_hit_df.head(n) , x ='Country Region' , y ='Confirmed' , size = 'Confirmed' , 
                     hover_name='Country Region', color = 'Country Region')
    fig.update_layout(title_text = 'Top ' + str(n) +" Hit" + " Countries" , title_font_size = 20)
    return fig.show()
interact(worst_hit_scatter , n = (1,10))

interactive(children=(IntSlider(value=5, description='n', max=10, min=1), Output()), _dom_classes=('widget-int…

<function __main__.worst_hit_scatter(n)>

## Worst Hit Countries Based On Number Of Deaths (Scatter Plot)

In [6]:
worst_hit_deaths  = country_df.sort_values(by = 'Deaths' , ascending = False)
def worst_hit_deaths_scatter(n):
    fig = px.scatter(worst_hit_deaths.head(n) , x = 'Country Region' , y = 'Deaths' ,size = 'Deaths' , 
                     color  = 'Country Region' , hover_name='Country Region' )
    return fig.show()
interact(worst_hit_deaths_scatter, n = (1,10))

interactive(children=(IntSlider(value=5, description='n', max=10, min=1), Output()), _dom_classes=('widget-int…

<function __main__.worst_hit_deaths_scatter(n)>

## Confirmed vs Death Numbers 

In [65]:
def plot_cases_for_country(Country):
    labels = ['Confirmed' , 'Deaths']
    colors = ['blue' , 'red']
    node_size = [6 , 8]
    line_size = [4 , 5]
    df_list = [confirmed_df , death_df]
    fig = go.Figure()
    for i, df in enumerate(df_list):
        if Country == 'World' or Country == 'world':
            x_data = np.array(list(df.iloc[:, 4:].columns))
            y_data = np.sum(np.asarray(df.iloc[:,4:]) , axis = 0)
        else:
            x_data = np.array(list(df.iloc[:,4:].columns))
            y_data = np.sum(np.asarray(df[df['Country/Region'] == Country].iloc[:,4:]) , axis = 0)
        fig.add_trace(go.Scatter(x=x_data, y=y_data, mode='lines+markers',
        name=labels[i],
        line=dict(color=colors[i], width=line_size[i]),
        ));
    fig.show()
interact(plot_cases_for_country , Country = 'World')

interactive(children=(Text(value='World', description='Country'), Output()), _dom_classes=('widget-interact',)…

<function __main__.plot_cases_for_country(Country)>

## World Plot

In [7]:
country_df = country_df.dropna(subset = ['Lat' , 'Long'])
world_map = folium.Map(location=[11,0], tiles="cartodbpositron", zoom_start=2, max_zoom = 6, min_zoom = 2)
for i in range(0 , len(country_df)):
    folium.Circle(
    location = [country_df.iloc[i]['Lat'] , country_df.iloc[i]['Long']],
    fill=True,
    radius=120,
    color='red',
    fill_color='indigo',
    tooltip = "<b>Country : </b>" + str(country_df.iloc[i]['Country Region']) + '<br>' + 
        " <b>Confirmed Cases : </b>" +str(country_df.iloc[i]['Confirmed'])+ '<br>'
    ).add_to(world_map)
world_map